In [24]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import seaborn as sns;
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree
import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.pipeline import Pipeline

def stdscaler(df):
    scaler = StandardScaler()
    scaler.fit(df)
    scX_df = scaler.transform(df)
    df=pd.DataFrame(scX_df, index =df.index,columns =df.columns  )
    return df
def kfoldtest(kfoldn,model,Xtrain_df,ytrain_df):
    kf=StratifiedKFold(n_splits=kfoldn)
    accu=[]
    auc=[]
    f1 =[]
    aab =[]
    dcd = []
    tim = []
    kf.get_n_splits(Xtrain_df,ytrain_df)
    
    for train_index, test_index in kf.split(Xtrain_df,ytrain_df):
        #print("TRAIN:", len(train_index), "TEST:", len(test_index))
        t1 =time.time()
        X_train, X_test = Xtrain_df.iloc[train_index], Xtrain_df.iloc[test_index]
        y_train, y_test = ytrain_df.iloc[train_index], ytrain_df.iloc[test_index]
        
        """
        if samplechoice == "SMOTE":
            smt = SMOTE()
            X_train,  y_train = smt.fit_sample(X_train , y_train)
        elif samplechoice == "NearMiss":
            nr = NearMiss()
            X_train,  y_train = nr.fit_sample(X_train , y_train)
        """    
    
        model.fit(X_train,y_train)
        try:
            accu.append(model.score(X_test,y_test))
        except:
            print("This model have no build-in score.")
            
        auc.append(roc_auc_score(y_test,model.predict_proba(X_test)[:,1]))
        f1.append(f1_score(y_test, model.predict(X_test).astype(int)))
        tim.append(time.time()-t1)
        cm1=confusion_matrix(test_y, clf.predict(test_data),labels = [1,0])
        print("cv result:")
        print(cm1)
        sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
        aab.append(sensitivity1)

        specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
        dcd.append(specificity1)
    return accu,auc,f1,aab,dcd,tim


sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})


In [25]:
file_name = 'creditcard 2.csv'
data =pd.read_csv(file_name)
del data['Time']
u=StandardScaler().fit_transform(np.array(data['Amount']).reshape(-1,1))
data['Amount']=pd.Series(u[:,0],index =data.index)
train_y = data['Class'].copy()
train_data =data.drop(columns='Class')

train_data,test_data,train_y,test_y = train_test_split(
    train_data, train_y, test_size=0.20, random_state=42)


In [26]:
samplechoice = "NearMiss"

if samplechoice == "SMOTE":
    smp = SMOTE()

elif samplechoice == "NearMiss":
    smp = NearMiss()
else:
    smp=None


In [27]:
model_holder = []
cvresult_holder = []
testroc_holder = []
testlabel_holder =[]

In [28]:



param_grid  = { "lr__C":np.logspace(-3,3,7),
               "lr__penalty":[None,"l2"],
               'lr__class_weight':[None,'balanced']
               
               
                }

model = Pipeline([
        ('sampling', smp),
        ('lr', LogisticRegression(max_iter=3000))
    ])


clf = GridSearchCV(model, param_grid, cv = 5, scoring = 'roc_auc',n_jobs=-1)
clf.fit(train_data,train_y)

bestfit =clf.best_estimator_
model_holder.append(bestfit)
"""cv"""
accu,auc,f1,aab,dcd,tim =kfoldtest(5,bestfit,train_data,train_y)
cvresult_holder.append(pd.DataFrame([["LogisticRegression",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



cv result:
[[   79    19]
 [  181 56683]]
cv result:
[[   78    20]
 [   54 56810]]
cv result:
[[   79    19]
 [   96 56768]]
cv result:
[[   78    20]
 [   97 56767]]
cv result:
[[   79    19]
 [  116 56748]]


In [29]:

model = Pipeline([
        ('sampling',smp),
        ('mlp', MLPClassifier(max_iter=3000))
    ])



param_grid  = { 'mlp__hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                'mlp__activation': ['tanh', 'relu'],
                'mlp__solver': ['sgd', 'adam'],
                'mlp__alpha': [0.0001, 0.05],
                'mlp__learning_rate': ['constant','adaptive'],
                
                }

clf = GridSearchCV(model, param_grid, cv = 5, scoring = 'roc_auc',n_jobs=-1)
clf.fit(train_data,train_y)
bestfit =clf.best_estimator_
model_holder.append(bestfit)
"""cv"""
accu,auc,f1,aab,dcd,tim =kfoldtest(5,bestfit,train_data,train_y)
cvresult_holder.append(pd.DataFrame([["MLP",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



cv result:
[[   98     0]
 [35985 20879]]
cv result:
[[   94     4]
 [25005 31859]]
cv result:
[[   92     6]
 [24326 32538]]
cv result:
[[   93     5]
 [33946 22918]]
cv result:
[[   96     2]
 [33525 23339]]


In [30]:
param_grid = {"dtc__max_depth": range(1,20)
              , "dtc__random_state":[7]}


model = Pipeline([
        ('sampling', smp),
        ('dtc', DecisionTreeClassifier(random_state = 0, 
                             max_features = "auto", class_weight = "balanced",
                             max_depth = None))
    ])




clf = GridSearchCV(model, param_grid, cv = 5, scoring = 'roc_auc',n_jobs=-1)
clf.fit(train_data,train_y)
bestfit =clf.best_estimator_
model_holder.append(bestfit)
"""cv"""
accu,auc,f1,aab,dcd,tim =kfoldtest(5,bestfit,train_data,train_y)
cvresult_holder.append(pd.DataFrame([["DecisionTree",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



cv result:
[[   95     3]
 [34364 22500]]
cv result:
[[   85    13]
 [19811 37053]]
cv result:
[[   88    10]
 [14609 42255]]
cv result:
[[   86    12]
 [18275 38589]]
cv result:
[[   92     6]
 [30129 26735]]


In [31]:
param_grid = {
    "gbc__loss":["deviance"],
    "gbc__learning_rate": [0.01,  0.05,  0.1, 0.2],
    "gbc__min_samples_split": np.linspace(0.1, 0.5, 3),
    "gbc__min_samples_leaf": np.linspace(0.1, 0.5, 3),
    "gbc__max_depth":[3,5,8],
    "gbc__max_features":["log2","sqrt"],
    "gbc__criterion": ["friedman_mse",  "mae"],
    "gbc__subsample":[0.5,  0.8,   1.0],
    "gbc__n_estimators":[10]
    }
param_grid = {
    
    "gbc__learning_rate": [0.01,  0.05,  0.1, 0.2],

    "gbc__max_depth":[3,5,8],

    "gbc__subsample":[0.5,  0.8,   1.0],
    "gbc__n_estimators":[10]
    }

model = Pipeline([
        ('sampling',smp),
        ('gbc', GradientBoostingClassifier()) ])

clf = GridSearchCV(model ,param_grid, cv=5, scoring = 'roc_auc',n_jobs=-1)

clf.fit(train_data,train_y)
bestfit =clf.best_estimator_
model_holder.append(bestfit)
"""cv"""
accu,auc,f1,aab,dcd,tim =kfoldtest(5,bestfit,train_data,train_y)
cvresult_holder.append(pd.DataFrame([["GradientBoosting",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



cv result:
[[   98     0]
 [48144  8720]]
cv result:
[[   93     5]
 [22728 34136]]
cv result:
[[   92     6]
 [24173 32691]]
cv result:
[[   93     5]
 [39426 17438]]
cv result:
[[   98     0]
 [38458 18406]]


In [32]:
param_grid = { 
    'rfc__n_estimators': [200, 500],

    'rfc__max_depth' :(3,5,8,10,20),

    }

model = Pipeline([
        ('sampling', smp),
        ('rfc', RandomForestClassifier(random_state=42)) ])

clf = GridSearchCV(model,param_grid, cv= 5,scoring = 'roc_auc',n_jobs=-1)
clf.fit(train_data,train_y)
bestfit =clf.best_estimator_
model_holder.append(bestfit)
"""cv"""
accu,auc,f1,aab,dcd,tim =kfoldtest(5,bestfit,train_data,train_y)
cvresult_holder.append(pd.DataFrame([["RandomForest",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



cv result:
[[   97     1]
 [50656  6208]]
cv result:
[[   91     7]
 [10212 46652]]
cv result:
[[   95     3]
 [27643 29221]]
cv result:
[[   93     5]
 [31072 25792]]
cv result:
[[   95     3]
 [34556 22308]]


In [33]:
bestfit

Pipeline(memory=None,
         steps=[('sampling',
                 NearMiss(n_jobs=None, n_neighbors=3, n_neighbors_ver3=3,
                          sampling_strategy='auto', version=1)),
                ('rfc',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=3, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=500, n_jobs=None,
                                        oob_score=False, random_state=42,
                                        verbose=0, warm_s

In [34]:

from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(max_iter=3000,random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
    voting='soft'
)


param_grid = { 
    'rfc__lr__C': np.logspace(-3,3,7), 
    'rfc__rf__n_estimators': [500],
    
    'rfc__rf__max_depth' : [20]
    }

model = Pipeline([
        ('sampling', smp),
        ('rfc', eclf) ])

clf = GridSearchCV(model,param_grid, cv= 5,scoring = 'roc_auc',n_jobs=-1)
clf.fit(train_data,train_y)
bestfit =clf.best_estimator_
model_holder.append(bestfit)
"""cv"""
accu,auc,f1,aab,dcd,tim =kfoldtest(5,bestfit,train_data,train_y)
cvresult_holder.append(pd.DataFrame([["MajorVoting",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



cv result:
[[   94     4]
 [46464 10400]]
cv result:
[[   90     8]
 [14226 42638]]
cv result:
[[   91     7]
 [13573 43291]]
cv result:
[[   90     8]
 [12278 44586]]
cv result:
[[   92     6]
 [15452 41412]]


In [35]:
result= pd.concat(cvresult_holder,axis=0)
result.to_csv('cvresult_{}sampling.csv'.format(str(samplechoice)))
result

,model names,CVaccuracy,CVaucroc,CVF1,CVsensitivity,CVspecificity,CVtime(s)
0,LogisticRegression,0.997705,0.901340,0.556792,0.802041,0.998087,1.832592
0,MLP,0.462933,0.701890,0.006167,0.965306,0.462623,4.259487
0,DecisionTree,0.587189,0.728695,0.007917,0.910204,0.587831,1.808477
0,GradientBoosting,0.394189,0.667475,0.005795,0.967347,0.391780,1.858169
0,RandomForest,0.458667,0.694699,0.007677,0.961224,0.457868,4.549060
0,MajorVoting,0.642827,0.766688,0.010758,0.932653,0.641274,5.533175


In [36]:
"""
fpr,tpr,label =testresult(clf,'MajorVoting',test_data,test_y)
testroc_holder.append([fpr,tpr])
testlabel_holder.append(label)


plt.figure(figsize=(11.7,8.27))
for i in range(len(muholder)):
    fpr, tpr =testroc_holder[i][0],testroc_holder[i][1]
    plt.plot(fpr, tpr,label=testlabel_holder[i])
    
plt.plot([0,1],[0,1],"--",color ='r')
plt.xlabel('1-specificity')
plt.legend()
plt.show()
plt.savefig('ruc_curve_{}sampling.csv'.format(str(samplechoice)))

"""
#from sklearn.externals import joblib
#joblib.dump(grid.best_estimator_, 'filename.pkl')

'\nfpr,tpr,label =testresult(clf,\'MajorVoting\',test_data,test_y)\ntestroc_holder.append([fpr,tpr])\ntestlabel_holder.append(label)\n\n\nplt.figure(figsize=(11.7,8.27))\nfor i in range(len(muholder)):\n    fpr, tpr =testroc_holder[i][0],testroc_holder[i][1]\n    plt.plot(fpr, tpr,label=testlabel_holder[i])\n    \nplt.plot([0,1],[0,1],"--",color =\'r\')\nplt.xlabel(\'1-specificity\')\nplt.legend()\nplt.show()\nplt.savefig(\'ruc_curve_{}sampling.csv\'.format(str(samplechoice)))\n\n'

In [37]:
from sklearn.externals import joblib

for i in range(len(model_holder)):
    joblib.dump(model_holder[i], 'model{}.pkl'.format(samplechoice+str(i)))
    
#joblib.load('best_tfidf.pkl') load function.

In [38]:
model_holder[0]

Pipeline(memory=None,
         steps=[('sampling',
                 NearMiss(n_jobs=None, n_neighbors=3, n_neighbors_ver3=3,
                          sampling_strategy='auto', version=1)),
                ('lr',
                 LogisticRegression(C=0.001, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=3000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)